In [1]:
import sys
sys.path.append("../..")
from src.service.provider import ProviderService
from src.rag.hg_parent_retriever import ConfigParentRAG, HugFaceParentRAG
provider = ProviderService()

In [2]:
from datasets import load_dataset

DATA_REPO = "BroDeadlines/TEST.TDT.edu_tdt_proposition_data"
SPLIT = "INDEX.medium_index_TDT_clean"

dataset = load_dataset(DATA_REPO, split=SPLIT)

In [3]:
config:ConfigParentRAG = {}

config["vec_index"] = "vec-sentence-propositon_medium_edu_tdt"
config["txt_index"] = "text-sentence-propositon_medium_edu_tdt"
config['vec_weight'] = 0.5
config['txt_weight'] = 0.5
config['total_k'] = 4
config['llm'] = "gemini-1.5-flash"

In [4]:
rag = HugFaceParentRAG(provider=provider, config=config, text_corpora=dataset)

In [7]:
question = "Em đi thi mà đề thi không nằm trong đề cương môn học thì em phải làm sao"
question = "Tiền tạm thu khi nhập học sẽ được sử dụng như thế nào có được cấn trừ qua học kỳ kế tiếp không"
res = rag.answer(question=question)

res

page_content='Nhà trường sẽ tiến hành hậu kiểm hồ sơ\xa0nhập học sau khi thí sinh đã nộp hồ sơ nhập học (bản giấy).' metadata={'doc_id': 'ebe5e14e-13cc-11ef-b548-0242ac1c000c'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

page_content='Những môn học được quy định không tham gia tính điểm trung bình sẽ không đưa vào công thức này để tính' metadata={'doc_id': '573ca1c3-1b6d-11ef-a755-d38426455a06'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

page_content='Thẻ tạm hoặc phiếu báo mất thẻ sinh viên không được sử dụng cho bất kỳ lý do nào khác.' metadata={'doc_id': '573ca190-1b6d-11ef-a755-d38426455a06'}


...

page_content='Số tiền còn nợ bằng học phí phải nộp của học kỳ trừ đi số tiền đã nộp (học phí tạm thu).' metadata={'doc_id': '3369b8e1-1b47-11ef-a755-d38426455a06'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

page_content='Phần chênh lệch học phí của học kỳ 1 sẽ được cấn trừ sang học kỳ 2 (trong cột nợ kỳ trước).' metadata={'doc_id': '3369b8e1-1b47-11ef-a755-d38426455a06'}
page_content='Môn học thay thế được sử dụng khi một môn học có trong CTĐT nhưng nay không còn tổ chức giảng dạy nữa và được thay thế bằng một môn học khác đang còn tổ chức giảng dạy.' metadata={'doc_id': '3369b8de-1b47-11ef-a755-d38426455a06'}


Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

{'answer': 'Tiền tạm thu khi nhập học sẽ được cấn trừ vào học phí của học kỳ 1. Phần chênh lệch học phí sẽ được cấn trừ sang học kỳ 2. \n',
 'retrieved_docs': [Document(page_content='Nhà trường sẽ tiến hành hậu kiểm hồ sơ\xa0nhập học sau khi thí sinh đã nộp hồ sơ nhập học (bản giấy).', metadata={'doc_id': 'ebe5e14e-13cc-11ef-b548-0242ac1c000c'}),
  Document(page_content='Những môn học được quy định không tham gia tính điểm trung bình sẽ không đưa vào công thức này để tính', metadata={'doc_id': '573ca1c3-1b6d-11ef-a755-d38426455a06'}),
  Document(page_content='Thẻ tạm hoặc phiếu báo mất thẻ sinh viên không được sử dụng cho bất kỳ lý do nào khác.', metadata={'doc_id': '573ca190-1b6d-11ef-a755-d38426455a06'}),
  Document(page_content='Số tiền còn nợ bằng học phí phải nộp của học kỳ trừ đi số tiền đã nộp (học phí tạm thu).', metadata={'doc_id': '3369b8e1-1b47-11ef-a755-d38426455a06'}),
  Document(page_content='Phần chênh lệch học phí của học kỳ 1 sẽ được cấn trừ sang học kỳ 2 (trong cột nợ

In [2]:

ground = " Đầu năm khi làm thủ tục nhập học, sinh viên sẽ nộp học phí tạm thu của học kỳ 1 năm nhất và các khoản chi phí khác. Sinh viên có thể kiểm lại học phí của học kỳ 1 Nhà Trường đã công bố, Số tiền còn nợ = Học phí phải nộp của học kỳ - Số tiền đã nộp (Học phí tạm thu). Phần chênh lệch học phí của học kỳ 1 sẽ được cấn trừ sang học kỳ 2 (trong cột nợ kỳ trước)."
res = "Tiền tạm thu khi nhập học sẽ được cấn trừ vào học phí của học kỳ 1. Phần chênh lệch học phí sẽ được cấn trừ sang học kỳ 2. \n"



In [2]:
from nltk.translate.bleu_score import sentence_bleu

reference = [ground.strip().split(" ")]
candidate = res.strip().split(" ")
score = sentence_bleu(reference, candidate)
print(score)
score1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
score1

0.0670546934524819


0.11090996875656291

In [3]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

scores = scorer.score(ground, res)

scores

{'rouge1': Score(precision=0.9777777777777777, recall=0.34108527131782945, fmeasure=0.5057471264367815),
 'rouge2': Score(precision=0.75, recall=0.2578125, fmeasure=0.38372093023255816),
 'rougeL': Score(precision=0.8888888888888888, recall=0.31007751937984496, fmeasure=0.4597701149425287)}

In [6]:
scores['rougeL'].precision

0.8888888888888888

In [2]:
import numpy as np
a = [3, 4, 5]

a= np.array(a)

np.median(a)

4.0

In [3]:
a.mean()

4.0